In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root = '/content/drive/MyDrive/Hoc_Tap/CS410_MangNeural/Final_Project'

# Tổng quan:
- Ở trong bài này nhóm chúng tôi sẽ áp dụng thuật toán Genetic Algorithm để tối ưu hóa đơn mục tiêu.
- Mục tiêu mà nhóm chúng tôi đã chọn để tối ưu hóa là ***test-accuracy*** trên 3 tập dữ liệu **cifar10-valid**, **cifar100**, **ImageNet16-200** tại **200** training epochs, giá trị của mục tiêu được đánh giá bằng hàm benchmark NAS-Bench-201.
- Mục tiêu trong bài này là nhóm chúng tôi sẽ phát sinh ra các kiến trúc ngẫu nhiên, sau đó áp dụng thuật toán GA để tối đa hóa ***test-accuracy*** nhằm tìm ra kiến trúc có độ chính xác trên tập dữ liệu test cao nhất trong NAS-Bench-201.
- Đồng thời nhóm cũng đã chạy thí nghiệm trên 31 random seed khác nhau nhằm tìm kích thước quần thể nhỏ nhất cần thiết (minimally required population sice - MRPS) để thuật toán GA có thể tìm ra kiến trúc tốt nhất trong NAS-Bench-201.
- Kết quả thực nghiệm trả về kích thước quần thể nhỏ nhất cần thiết (MRPS) là **320**. Với số lượng cá thể là 320, thuật toán GA đã tìm được kiến trúc tốt nhất với ***test-accuracy*** là 91.52, đồng thời đây cũng kiến trúc tốt nhất mà nhóm đã vét cạn trong NAS-Bench-201.

# Khai báo thư viện

In [ ]:
import numpy as np
import json

# NAS_Benchmark_201

## Setup

1. Tải dữ liệu của NAS-Bench-201 tại link:
  *   [NATS-tss-v1_0-3ffb9-simple](https://drive.google.com/drive/folders/1FDXJ-nhBFHfm7pcA-wFnsBrmSFwTmK2h?usp=sharing)
2. Cài đặt các thư viện cần thiết:
  *   nats_bench (pip install nats_bench)

In [ ]:
!pip install nats_bench

## Khai báo thư viện

In [ ]:
from nats_bench import create

## Khởi tạo API
* from nats_bench import create
* api = create('path_data', 'tss', fast_mode=True)

  

In [ ]:
api = create(f'{root}/NAS_Benchmark/datasets/NATS-tss-v1_0-3ffb9-simple', 'tss', fast_mode=True)
api.verbose = False

[2021-12-31 01:04:10] Try to create the NATS-Bench (topology) api from /content/drive/MyDrive/Hoc_Tap/CS410_MangNeural/Final_Project/NAS_Benchmark/datasets/NATS-tss-v1_0-3ffb9-simple with fast_mode=True
[2021-12-31 01:06:02] Create NATS-Bench (topology) done with 0/15625 architectures avaliable.


## Mã hóa kiến trúc
-------------------------------------------------------------------------------
* Các kiến trúc được biểu diễn theo định dạng:

  '|operation\~0|+|operation\~0|operation\~1|+|operation\~0|operation\~1|operation~2|'

  với operation có thể là 1 trong 5 loại:
    * 'none'
    * 'skip_connect'
    * 'nor_conv_1x1'
    * 'nor_conv_3x3'
    * 'avg_pool_3x3'

* Example:
  * arch = '|none\~0|+|skip_connect\~0|none\~1|+|nor_conv_3x3\~0|nor_conv_1x1\~1|avg_pool_3x3\~2|' 


In [ ]:
def encode_arch(pop):
  arch_lst = []
  operation_matrix = ['none', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
  pop = np.reshape(pop, (-1, 6))
  for arch in pop:
    arch = f'|{operation_matrix[arch[0]]}~0|+|{operation_matrix[arch[1]]}~0|{operation_matrix[arch[2]]}~1|+|{operation_matrix[arch[3]]}~0|{operation_matrix[arch[4]]}~1|{operation_matrix[arch[5]]}~2|'
    arch_lst.append(arch)

  return np.array(arch_lst)

## Hàm lấy giá trị hàm mục tiêu

-------------------------------------------------------------------------------
* Tổng cộng có 15625 kiến trúc trong NAS-Bench-201. Các thông tin được truy suất bằng số thứ tự của kiến trúc đó trong dữ liệu. Do đó, chúng ta cần chuyển từ dạng biểu diễn dạng 'string' sang số thứ tự (int).
* Syntax:
  
  idx = api.query_index_by_arch('kiến trúc biểu diễn ở dạng string')
* Example:

  idx = api.query_index_by_arch('|none\~0|+|skip_connect\~0|none\~1|+|nor_conv_3x3\~0|nor_conv_1x1\~1|avg_pool_3x3\~2|')
-------------------------------------------------------------------------------
* Syntax để lấy các giá trị accuracy (training, validation, testing)

  api.get_more_info(index, dataset=dataset, hp=hp)

  Trong đó:
    * index -> Vị trí của kiến trúc trong bộ benchmark (kiểu int)
    * dataset -> Bộ dữ liệu để huấn luyện và đánh giá kiến trúc. Có thể chọn 1 trong 3 giá trị (kiểu string):
      * 'cifar10-valid'
      * 'cifar100'
      * 'ImageNet16-120'
    * hp -> Số training epochs. Có thể chọn trong 2 giá trị (kiểu string):
      * '12'
      * '200'
* Kết quả trả về là 1 dictionary. Chúng ta có thể dễ dàng lấy những thông tin như mong muốn.

In [ ]:
# def get_obj_value( arch, dataset, hp, obj_value ):   
#     idx = api.query_index_by_arch(arch)
#     info = api.get_more_info(idx, dataset=dataset, hp=str(hp), is_random=False)
#     acc_value = np.round(info[obj_value], 2)
    
#     return acc_value

In [ ]:
def get_obj_value( arch, dataset, hp, obj_value ):
  with open(f'{root}/NAS_Benchmark/datasets/NAS_201_{dataset}_{hp}.txt', 'r') as fi:
    archs = fi.readlines()
    idx = api.query_index_by_arch(arch)
    arch_info = json.loads(archs[idx].strip())
    acc_value = arch_info[obj_value]
  
  return acc_value

## Hàm lấy kiến trúc tốt nhất

In [ ]:
def get_best_arch( dataset, hp, obj_value ):
  max_acc = 0.0

  with open(f'{root}/NAS_Benchmark/datasets/NAS_201_{dataset}_{hp}.txt', 'r') as fi:
    archs = fi.readlines()
    for arch in archs:
      arch = arch.strip()
      arch_info = json.loads(arch)
      test_accuracy = arch_info['test_accuracy']
      if test_accuracy > max_acc:
        max_acc = test_accuracy
        best_arch_index = arch_info['arch_index']
    
  return best_arch_index, max_acc

# Thuật toán Genetic Algorithm

## Xây dựng quần thể

In [ ]:
def initialize_population( num_individuals ):
    """
    Khởi tạo quần thể gồm num_individuals cá thể..
    
    Arguments:
    num_individuals -- Số lượng cá thể
    
    Returns:
    pop -- Ma trận (num_individuals) chứa quần thể mới được khởi tạo ngẫu nhiên.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ###
    pop = np.random.randint(5, size=(num_individuals, 6))
    
    ### DỪNG CODE TẠI ĐÂY ###
    
    return pop

## Đánh giá độ thích nghi của quần thể

In [ ]:
def evaluate_population( pop, dataset, hp, obj_value ):
    """
    Hàm đánh giá tất cả cá thể trong quần thể.
    
    Arguments:
    pop -- Quàn thể cần được đánh giá.

    Returns:
    values -- Giá trị của tất cả các cá thể trong quần thể.
    """ 

    ### BẮT ĐẦU CODE TỪ ĐÂY ###
    arch_lst = encode_arch(pop)
    values = np.array([get_obj_value(ind, dataset, hp, obj_value) for ind in arch_lst])
    
    ### DỪNG CODE TẠI ĐÂY ###
    
    return values

## So sánh độ thích nghi giữa hai cá thể

In [ ]:
def better_fitness( fitness_1, fitness_2, maximization=True ):
    """
    Hàm so sánh độ thích nghi của 2 cá thể.
    
    Arguments:
    fitness_1 -- Độ thích nghi của cá thể 1.
    fitness_2 -- Độ thích nghi của cá thể 2.
    maximization -- Biến boolean cho biết bài toán đang giải thuộc dạng tối đa hoá (mặc định) hay không
    
    Returns:
    True nếu fitness_1 tốt hơn fitness_2. Ngược lại, trả về False.
    """
    
    if maximization:
        if fitness_1 > fitness_2:
            return True
    else:
        if fitness_1 < fitness_2:
            return True
        
    return False

## Tourament selection

In [ ]:
def tournament_selection( pop, pop_fitness, selection_size, tournament_size):
    """
    Hàm chọn lọc cạnh tranh.
    
    Arguments:
    pop -- Quần thể để thực hiện phép chọn lọc.
    pop_fitness -- Mảng 1 chiều chứa giá trị (độ thích nghi) của từng cá thể trong quần thể.
    selection_size -- Số lượng cá thể sẽ được chọn.
    tournament_size -- Kích thước của tournament: Số lượng các cá thể được so sánh với nhau mỗi lần.
    
    Returns:
    selected_indices -- Chỉ số của những cá thể trong quần thể pop được chọn. Chỉ số có thể được lặp lại.
    """
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    num_individuals = len(pop)
    indices = np.arange(num_individuals)
    selected_indices = []
    
    while len(selected_indices) < selection_size:
        # Đảo ngẫu nhiên thứ tự các cá thể trong quần thể.
        np.random.shuffle(indices)
        
        for i in range(0, num_individuals, tournament_size):
          best_idx = i
          for idx in range(1, tournament_size):
            if better_fitness(pop_fitness[indices[i+idx]], pop_fitness[indices[best_idx]]):
              best_idx = i + idx
          selected_indices.append(indices[best_idx])
    
    selected_indices = np.array(selected_indices)        

    ### DỪNG CODE TẠI ĐÂY ###
    
    return selected_indices

## Lai ghép quần thể

In [ ]:
def variation( pop ):
    """
    Hàm biến đổi tạo ra các cá thể con.
    
    Arguments:
    pop -- Quàn thể hiện tại.

    Returns:
    offspring -- Quần thể chứa các cá thể con được sinh ra.
    """  
    
    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    num_individuals = len(pop)
    indices = np.arange(num_individuals)
    # Đảo ngẫu nhiên thứ tự các cá thể trong quần thể
    np.random.shuffle(indices)
    offspring = []
    
    for i in range(0, num_individuals, 2):
        idx1 = indices[i]
        idx2 = indices[i+1]
        offspring1 = list(pop[idx1])
        offspring2 = list(pop[idx2])
        
        # Cài đặt phép lai đồng nhất uniform crossover. 
        # Không cần cài đặt đột biến mutation.
        for idx in range(0, 6):
          r = np.random.rand()
          if r < 0.5:
            temp = offspring2[idx]
            offspring2[idx] = offspring1[idx]
            offspring1[idx] = temp

        offspring.append(offspring1)
        offspring.append(offspring2)


    ### DỪNG CODE TẠI ĐÂY ###
    
    offspring = np.array(offspring)
    return offspring

## Hàm kiểm tra hội tụ của quần thể

In [ ]:
def check_convergence(pop_fitness):
  if len(np.unique(pop_fitness)) == 1:
    return True
    
  return False

## Hàm kiểm tra tối ưu của quần thể

In [ ]:
def check_optimal(best_fitness, pop_fitness):
  if best_fitness in pop_fitness:
    return True

  return False

## Thuật toán POPOP 

In [ ]:
def popop(num_individuals, dataset, hp, obj_value):
    """
    Hàm cài đặt thuật giải di truyền theo các bước P->O->(P+O)->P
    
    Arguments:
    num_individuals -- Số lượng cá thể trong quần thể.
    num_generations -- Số thế hệ thuật toán sẽ chạy.

    Returns:
    In ra quần thể ở thế hệ cuối cùng và giá trị của từng cá thể.
    """ 

    ### BẮT ĐẦU CODE TỪ ĐÂY ### 
    # Khởi tạo quần thể
    pop = initialize_population(num_individuals)
    original_pop = np.copy(pop)
    pop_fitness = evaluate_population(pop, dataset, hp, obj_value)
    original_pop_fitness = np.copy(pop_fitness)
    
    # Sử dụng tournament_size 4 và selection_size là bằng kích thước quần thể
    selection_size = len(pop)
    tournament_size = 4
    num_generations = 0

    while not check_convergence(pop_fitness):
        num_generations += 1
        # Tạo ra các cá thể con và đánh giá chúng
        offspring = variation(pop)
        offspring_fitness = evaluate_population(offspring, dataset, hp, obj_value)
        
        # Tạo ra quần thể pool gồm quần thể hiện tại pop và offspring
        pool = np.vstack((pop,offspring))
        pool_fitness = np.hstack((pop_fitness, offspring_fitness))

        # Thực hiện tournament selection trên quần thể pool 
        pool_indices = tournament_selection(pool, pool_fitness, selection_size, tournament_size)

        # Thay thế quần thể hiện tại bằng những cá thể được chọn ra từ pool.
        pop = pool[pool_indices, :]
        pop_fitness = pool_fitness[pool_indices]

    ### DỪNG CODE TẠI ĐÂY ###

    return [original_pop, original_pop_fitness, pop, pop_fitness, num_generations]

## Hàm ghi chép quá trình tìm kiếm MRPS của quần thể

In [ ]:
def record(lines):
  record_file = open(f'{root}/report/MRPS_record.txt', 'a')
  for line in lines:
    record_file.write(line)
    record_file.write('\n')
    print(line)

In [ ]:
def MRPS_record(type_record=None, dataset=None, hp=None, obj_value=None, pop_size=None, seed=None, s_flag=None, mrps=None, upper_bound=None):
  lines = []
  
  if type_record == 1:
    line1 = '|--------------------------------MRPS RECORD--------------------------------|'
    line2 = '|-----------------------------------BEGIN-----------------------------------|\n'
    lines.append(line1)
    lines.append(line2)
    record(lines)
  elif type_record == 2:
    line = '\n|-----------------------------------DONE-----------------------------------|'
    lines.append(line)
    record(lines) 
  elif type_record == 3:
    line = f'- Algorithm = Genetic Algorithm, Objective value = {obj_value}, Dataset = {dataset}, Checkpoint = {hp}'
    lines.append(line)
    record(lines)
  elif type_record == 4:
    line = f'\t- Finding the upper bound of MRPS'
    lines.append(line)
    record(lines)
  elif type_record == 5:
    line = f'\t\t- Population size = {pop_size}'
    lines.append(line)
    record(lines)
  elif type_record == 6:
    line = f'\t\t\t{seed}-th seed'
    lines.append(line)
    record(lines)
  elif type_record == 7:
    if s_flag == True:
      line = '\t\t\t\tThe best architecture has been found!'
    else:
      line = "\t\t\t\tCan't find the best architecture!"
    lines.append(line)
    record(lines)
  elif type_record == 8:
    line = f'\n\t- Finding the MRPS'
    lines.append(line)
    record(lines)
  elif type_record == 9:
    line = f'\n\t- Minimally required population size to find the best architecture: {mrps}'
    line1 = '\tSaving...'
    line2 = '\tSaved record!\n'
    lines.append(line)
    lines.append(line1)
    lines.append(line2)
    record(lines)
  elif type_record == 10:
    line = f"\n\t- Can't find the minimally required population size to find the best architecture"
    line1 = '\tSaving...'
    line2 = '\tSaved record!\n'
    lines.append(line)
    lines.append(line1)
    lines.append(line2)
    record(lines)
  elif type_record == 11:
    line = f'\t\t The upper bound of MRPS: {upper_bound}'
    lines.append(line)
    record(lines)

## Hàm đánh giá thuật toán sGA với 10 random seed khác nhau

In [ ]:
def evaluate_sGA(num_individuals, dataset, hp, obj_value, highest_obj_value):
  MRPS_record(type_record=5, pop_size=num_individuals)
  for i in range(31):
    MRPS_record(type_record=6, seed=i)
    np.random.seed(i)
    pop_fitness = popop(num_individuals, dataset, hp, obj_value)[3]
    if not check_optimal(highest_obj_value, pop_fitness):
      MRPS_record(type_record=7, s_flag=False)
      return False
    MRPS_record(type_record=7, s_flag=True)

  return True

## Hàm tìm kiếm cận trên của population size

In [ ]:
def upper_bound(dataset, hp, obj_value, highest_obj_value, cant_solve):
  MRPS_record(type_record=4)
  upper_popSize = 4
  while not evaluate_sGA(upper_popSize, dataset, hp, obj_value, highest_obj_value):
    upper_popSize *= 2

    if upper_popSize > 15624:
      cant_solve = True
      break
  MRPS_record(type_record=11, upper_bound=upper_popSize)
  return upper_popSize, cant_solve

## Hàm tìm kiếm MRPS của quần thể

In [ ]:
def MRPS(upper_popSize, dataset, hp, obj_value, highest_obj_value):
  MRPS_record(type_record=8)
  lower_popSize = upper_popSize // 2
  while (upper_popSize - lower_popSize) / upper_popSize > 0.1:
    popSize = (upper_popSize + lower_popSize) // 2
    flag = evaluate_sGA(popSize, dataset, hp, obj_value, highest_obj_value)
    if flag:
      upper_popSize = popSize
    else:
      lower_popSize = popSize

    if (upper_popSize - lower_popSize) <= 2:
      break

  MRPS_record(type_record=9, mrps=upper_popSize)
  return upper_popSize

## Bisection

In [ ]:
def bisection(dataset, hp, obj_value, highest_obj_value):
  MRPS_record(type_record=3, dataset=dataset, hp=hp, obj_value=obj_value)
  cant_solve = False
  upper_popSize, cant_solve = upper_bound(dataset, hp, obj_value, highest_obj_value, cant_solve)

  if cant_solve:
    MRPS_record(type_record=10)
    return None

  mrps = MRPS(upper_popSize, dataset, hp, obj_value, highest_obj_value)

  return mrps

# Tối ưu hóa kiến trúc 

  * dataset -> Bộ dữ liệu để huấn luyện và đánh giá kiến trúc. Có thể chọn 1 trong 3 giá trị (kiểu string):
    * 'cifar10-valid'
    * 'cifar100'
    * 'ImageNet16-120'
  * hp -> Số training epochs. Có thể chọn trong 2 giá trị (kiểu string):
    * '12'
    * '200'
  * obj_value -> Giá trị hàm mục tiêu cần tối ưu
    * 'train-accuracy'
    * 'valid-accuracy'
    * 'test-accuracy'

## Chạy bisection tìm kích thước quần thể nhỏ nhất cần thiết để tìm kiến trúc tối ưu nhất

In [ ]:
datasets = ['cifar10-valid', 'cifar100', 'ImageNet16-120']
hp = 200
# obj_value = 'test-accuracy'
obj_value = 'test_accuracy'

In [ ]:
exp_results = {}
MRPS_record(type_record=1)

for dataset in datasets:
  temp = {}

  best_arch_index, highest_test_accuracy = get_best_arch(dataset, hp, obj_value)
  mrps = bisection(dataset, hp, obj_value, highest_test_accuracy)

  temp['mrps'] = mrps
  temp['best_arch'] = api.arch(best_arch_index)
  temp['highest_test_accuracy'] = highest_test_accuracy

  exp_results[dataset] = temp
MRPS_record(type_record=2)

|--------------------------------MRPS RECORD--------------------------------|
|-----------------------------------BEGIN-----------------------------------|

- Algorithm = Genetic Algorithm, Objective value = test_accuracy, Dataset = cifar10-valid, Checkpoint = 200
	- Finding the upper bound of MRPS
		- Population size = 4
			0-th seed
				Can't find the best architecture!
		- Population size = 8
			0-th seed
				Can't find the best architecture!
		- Population size = 16
			0-th seed
				Can't find the best architecture!
		- Population size = 32
			0-th seed
				Can't find the best architecture!
		- Population size = 64
			0-th seed
				Can't find the best architecture!
		- Population size = 128
			0-th seed
				The best architecture has been found!
			1-th seed
				The best architecture has been found!
			2-th seed
				The best architecture has been found!
			3-th seed
				The best architecture has been found!
			4-th seed
				The best architecture has been found!
			5-th seed
				The 

## Tối ưu hóa kiến trúc dựa trên MRPS mới tìm 

In [ ]:
for dataset in datasets:
  [original_pop, original_pop_fitness, pop, pop_fitness, num_generations] = popop(exp_results[dataset]['mrps'], dataset, hp, obj_value)

  exp_results[dataset]['final_arch'] = encode_arch(pop[-1])[-1]
  exp_results[dataset]['final_test_accuracy'] = pop_fitness[-1]
  exp_results[dataset]['num_generations'] = num_generations

  print('\n\n------------------------------------------')
  print(f'Dataset: {dataset}')
  print(f"MRPS: {exp_results[dataset]['mrps']}")
  print("Original population: ", encode_arch(original_pop))
  print("Original population's fitness: ", original_pop_fitness)
  print("Final population", encode_arch(pop[-1]))
  print("Final population's fitness: ", pop_fitness)
  print("Num of generations:", num_generations)

  print(f"\nThe best architecture: {exp_results[dataset]['best_arch']}")
  print(f"The highest test accuracy: {exp_results[dataset]['highest_test_accuracy']}")

out_file = open(f"{root}/report/SOOP_results.json", "w")
json.dump(exp_results, out_file, indent = 6)
out_file.close()



------------------------------------------
Dataset: cifar10-valid
MRPS: 320
Original population:  ['|none~0|+|avg_pool_3x3~0|skip_connect~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_1x1~2|'
 '|none~0|+|nor_conv_1x1~0|avg_pool_3x3~1|+|nor_conv_1x1~0|nor_conv_1x1~1|skip_connect~2|'
 '|nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_1x1~2|'
 '|skip_connect~0|+|avg_pool_3x3~0|none~1|+|none~0|nor_conv_3x3~1|none~2|'
 '|none~0|+|nor_conv_3x3~0|skip_connect~1|+|none~0|skip_connect~1|none~2|'
 '|none~0|+|nor_conv_1x1~0|nor_conv_1x1~1|+|nor_conv_1x1~0|nor_conv_3x3~1|skip_connect~2|'
 '|nor_conv_3x3~0|+|skip_connect~0|nor_conv_1x1~1|+|avg_pool_3x3~0|none~1|nor_conv_1x1~2|'
 '|nor_conv_1x1~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|avg_pool_3x3~1|skip_connect~2|'
 '|nor_conv_1x1~0|+|none~0|skip_connect~1|+|none~0|nor_conv_1x1~1|skip_connect~2|'
 '|nor_conv_1x1~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|none~0|avg_pool_3x3~1|skip_connect~2|'
 '|nor_conv_3x3~0|+|no